In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw08.ipynb")

# Homework 8 – Advanced Visualization Techniques ✈️

## Data 94, Spring 2021

This homework is due on **Tuesday, April 20th, 2021. While it is due after the quiz, it is in your best interest to complete it before the quiz.** You must submit the assignment to Gradescope. Submission instructions can be found at the bottom of this notebook. See the [syllabus](http://data94.org/syllabus/#late-policy-and-extensions) for our late submission policy.

In [ ]:
# Run this cell, don't change anything.
!pip install kaleido

In [ ]:
# Run this cell, don't change anything.

from datascience import *
import numpy as np
Table.interactive_plots()
import copy
import plotly.io
from IPython.display import display, Image
np.set_printoptions(suppress=True)
import warnings
warnings.filterwarnings('ignore')

def save_and_show(fig, path):
    if fig == None:
        print('Error: Make sure to use the argument show = False above.')
    plotly.io.write_image(fig, path)
    display(Image(path))

## The Data

<img src = 'images/sfo.jpg' width = 600>

In this homework, we'll explore flight data from San Francisco International Airport (SFO).

## Disclaimer

When creating graphs – both standard graphs and maps – in this assignment, there are two things you will always have to do that we didn't talk about in lecture:
1. Assign the graph to a variable name. (As in previous assignments, we will tell you which variable names to use.)
2. Use the argument `show = False` in your graphing method, in addition to the other arguments you want to use.

These steps are **required** in order for your work to be graded. The distinction is subtle, since you will see the same visual output either way. See below for an example.

<b style="color:green;">Good:</b>

```py
fig_5 = table.sort('other_column').barh('column_name', show = False)
```

<b style="color:red;">Bad:</b>

```py
table.sort('other_column').barh('column_name')
```

Also note that most of this homework will be graded manually by us rather than being graded by an autograder.

## Question 1 – Big 3 Airlines

We'll start by looking at the table `sfo_activity`. The dataset was downloaded directly from [flysfo.com](https://data.sfgov.org/Transportation/Air-Traffic-Passenger-Statistics/rkru-6vcg), SFO's website.

Run the cell below to load it in.

In [ ]:
sfo_activity = Table.read_table('data/Air_Traffic_Passenger_Statistics.csv')
sfo_activity

The table has many columns, most of which are not useful for us. Let's start by cutting down the table so that it is in a more meaningful format for us.

Run the cell below. You don't need to change the cell at all, but you should understand how the code in it works.

In [ ]:
sfo_three = sfo_activity.select('Activity Period', 'Operating Airline', 'Activity Type Code', 'Passenger Count') \
            .where('Operating Airline', are.contained_in(['American Airlines', 'Delta Air Lines', 'United Airlines'])) \
            .where('Activity Period', are.above(201312)) \
            .group(['Activity Period', 'Operating Airline'], np.sum) \
            .select('Activity Period', 'Operating Airline', 'Passenger Count sum') \
            .relabeled('Passenger Count sum', 'Number of Passengers')

sfo_three

Our condensed table, `sfo_three`, contains information about the Big 3 US airlines – American, Delta, and United. The larger table `sfo_activity` separated passenger counts by type of transit (arriving and departing), destination (Domestic vs. International), and terminal, while `sfo_three` aggregates all of this information into one row per airline per month.

**For example, the first row of `sfo_three` tells us that 227423 passengers traveled on American Airlines to or from SFO in January 2014. (`sfo_three` only contains information about January 2014 and onwards.)**

There's another issue with `sfo_three`: the `'Activity Period'` column is formatted strangely. It contains dates as integers; this will be problematic when we go to create visualizations as our visualization library will interpret the difference between November 2016 (`201611`) and December 2016 (`201612`) as being different than the difference between December 2016 (`201612`) and January 2017 (`201701`).

That's not a problem – we can convert the `'Activity Period'` column into a format that Python understands as being a date. This way, when we go to create line plots, the x-axis will be set correctly.

Again, run the cell below. You should understand how the code in the cell works.

In [ ]:
def convert_activity_period(date):
    # This condition exists solely to safeguard against running this cell multiple times
    if isinstance(date, str) and '-' in date: 
        return date
    
    # Get year and month from YYYYMM (int)
    year = str(date)[:4]
    month = str(date)[4:]
    
    # Reformat to 'YYYY-MM' (str)
    return year + '-' + month

sfo_three = sfo_three.with_columns(
    'Activity Period', sfo_three.apply(convert_activity_period, 'Activity Period')
)

sfo_three

Great. Now we can proceed with creating visualizations.

### Question 1a

Our first goal will be to create a line plot of the number of passengers who traveled through SFO each month on each of the Big 3 airlines (we'll want three separate lines, one for each airline).

To do this, we will need to rearrange `sfo_three` so that it has columns for each airline, and only one row for each `'Activity Period'` (as opposed to the three it has now).

Below, assign `sfo_three_pivoted` to a table with four columns – `'Activity Period'`, `'American Airlines'`, `'Delta Air Lines'`, and `'United Airlines'` – and one row for each activity period. The numbers in the table should describe the number of passengers who traveled on a particular airline through SFO in a particular month. The first few rows of `sfo_three_pivoted` are shown below.


| Activity Period   |   American Airlines |   Delta Air Lines |   United Airlines |
|------------------:|--------------------:|------------------:|------------------:|
| 2014-01           |              227423 |            184953 |           1314840 |
| 2014-02           |              205019 |            173069 |           1188119 |
| 2014-03           |              240918 |            228384 |           1435920 |
| 2014-04           |              242020 |            231490 |           1517518 |
| 2014-05           |              246582 |            256089 |           1610137 |

_Hints:_ 
- _1. The name `sfo_three_pivoted` should tell you which table method to use._
- _2. You may also be wondering which function you need to provide as the fourth argument to the aforementioned method – most functions will work, since in `sfo_three` there is only one row for every combination of `'Activity Period'` and `'Operating Airline'`._
    - _We used `sum`; don't use `np.mean` since it may convert some numbers to scientific notation._

<!--
BEGIN QUESTION
name: q1a
points: 2
-->

In [ ]:
sfo_three_pivoted = ...
sfo_three_pivoted

In [ ]:
grader.check("q1a")

### Question 1b

Using the table you created in the previous subpart, assign `fig_1b` to a line plot describing the number of passengers traveling through SFO each month on Delta Air Lines. Your plot should match the example below, including the axis labels and title.

<img src = 'images/examples/1b.png' width = 800>

**Note:** Remember the disclaimer at the top of the assignment. Don't forget to set `show = False`.

In [ ]:
fig_1b = ...
fig_1b

<!-- BEGIN QUESTION -->

After creating your visualization above, run the following cell. You should see a picture of your graph. You must run this cell in order to get credit for this question.

<!--
BEGIN QUESTION
name: q1b
points: 2
manual: true
-->

In [ ]:
# Run this cell, don't change anything.
save_and_show(fig_1b, 'images/saved/1b.png')

<!-- END QUESTION -->



### Question 1c

Let's take things one step further. Now, assign `fig_1c` to a line plot describing the number of travelers through SFO each month for each of the three Big 3 airlines. Your plot should match the example below, including the axis labels and title.

<img src = 'images/examples/1c.png' width = 800>

**Note:** Remember the disclaimer at the top of the assignment. Don't forget to set `show = False`.

_Hint: You should only provide your plotting method with one column name; lines will automatically be drawn for all other columns. See the examples in Lecture 27._ 

In [ ]:
fig_1c = ...
fig_1c

<!-- BEGIN QUESTION -->

After creating your visualization above, run the following cell. You should see a picture of your graph. You must run this cell in order to get credit for this question.

<!--
BEGIN QUESTION
name: q1c
points: 2
manual: true
-->

In [ ]:
# Run this cell, don't change anything.
save_and_show(fig_1c, 'images/saved/1c.png')

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 1d

There are a few interesting trends evident in the line plot you created above. One, for instance, is that it seems like the month with the most travelers every year is usually either July or August, which makes sense as people tend to travel a lot in the summer.

In the cell below, answer the following questions.

1. Why are there far more passengers on United than on either of the other airlines? _(Hint: Go to the [Wikipedia article for SFO](https://en.wikipedia.org/wiki/San_Francisco_International_Airport) and look at the information under "Summary" on the right-hand side.)_

2. There is a particular month which seems to see the fewest travelers per year. Which month is it, and why do you think there are fewer travelers in this month than in any other month? _Hint: How many days are in each month? Also, remember that you can zoom in to certain regions of the plot._

3. What is the cause of the dramatic drop in early 2020?

<!--
BEGIN QUESTION
name: q1d
points: 3
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 2 – United

As we discovered in the previous question, United is by far the most popular airline at SFO.

<img src = 'images/united.jpg' width = 400>

Now we will look at data from the [US Bureau of Transportation Statistics](https://www.transtats.bts.gov/ONTIME/Departures.aspx) containing flight departure on-time statistics for all **domestic** flights on United from SFO in 2019. (We chose 2019 instead of 2020 so that seasonal travel trends are more evident – not many people traveled in the summer of 2020, for example.)

If you click the above hyperlink, you'll be able to enter the parameters you'd like; we chose `All Statistics`, `San Francisco, CA: San Francisco International (SFO)`, `United Airlines Inc. (UA)`, `All Months`, `All Days`, and `2019`, in that order. After entering your parameters and clicking `Submit`, you'll see a table; clicking the `csv` link above the table will download a `csv` that you can then use in a notebook.



Run the cell below to load in the resulting table. The extra arguments are there because the downloaded `csv` has a few unnecessary rows that need to be removed before we store the data as a table.

In [ ]:
united_raw = Table.read_table('data/Detailed_Statistics_Departures.csv', header = 6, skipfooter = 1, engine = 'python')
united_raw

Once again, the table has more columns than we need. Run the cell below to select only the columns we need and rename them to something slightly more convenient (`'Date'` instead of `'Date (MM/DD/YYYY)'`, for example).

In [ ]:
united = united_raw.select('Date (MM/DD/YYYY)', 'Flight Number', 'Destination Airport', 'Departure delay (Minutes)') \
                   .relabeled(['Date (MM/DD/YYYY)', 'Destination Airport', 'Departure delay (Minutes)'],
                              ['Date', 'Destination', 'Delay'])

united

This table, `united`, is much more manageable. It contains the date, flight number, destination, and delay (in minutes) of every domestic flight on United from SFO in 2019.

**Note:** You will not use the `sfo_three` or `sfo_three_pivoted` tables from this point forward.

### Question 2a

What are the most common destinations from SFO on United? Below, assign `fig_2a` to a bar chart that describes the number of flights on United from SFO to its top 10 destinations. Your bar chart should match the given example exactly.

<img src = 'images/examples/2a.png' width = 800>


**Note:** Remember the disclaimer at the top of the assignment. Don't forget to set `show = False`.

_Hint: You did this several times in Homework 7. Remember – `group`, `sort`, `take`, `barh`._

In [ ]:
fig_2a = ...
fig_2a

<!-- BEGIN QUESTION -->

After creating your visualization above, run the following cell. You should see a picture of your graph. You must run this cell in order to get credit for this question.

<!--
BEGIN QUESTION
name: q2a
points: 2
manual: true
-->

In [ ]:
# Run this cell, don't change anything.
save_and_show(fig_2a, 'images/saved/2a.png')

<!-- END QUESTION -->



### Question 2b

Cool! The results shouldn't be all that surprising, given that Newark (EWR), Chicago (ORD), Los Angeles (LAX), Denver (DEN), IAH (Houston), and IAD (Washington-Dulles) are all [hubs for United](https://en.wikipedia.org/wiki/United_Airlines#Hubs). The other airports in the top 10 are in popular West Coast destinations – Las Vegas (LAS), Seattle (SEA), San Diego (SAN), and Portland (PDX).

Up until now, the bar graphs we've drawn have all had the same colors by default. That's boring – let's look at how to change the color of our bars from blue to something else.

We'll start by making a copy of your `fig_2a` from the last subpart. Run the following cell.

In [ ]:
fig_2b = copy.deepcopy(fig_2a)

`fig_2b`, like `fig_2a`, is of type `Figure`. All figures have a method, `update_traces`, which allow us to change the figure's visual properties. To change bar colors, you should do the following:

Your job: Call `.update_traces(...)` on `fig_2b` with a single argument, `marker`. The value of the `marker` argument should be a **dictionary** with a single key-value pair; the key should be `'color'` and the value should be the color you want as a string (like `'green'`, `'orange'`, `'red'`, etc).

Your code should fit on a single line, and should be in the following format. (The above paragraph tells you what should go in each blank; you just need to match the description to the syntax. This is the only line of code you need to write for this question.)

<div align=center>

```py
___.___(___ = {___: ___})
```
    
</div>

Start by using `'green'` as your color. Once you get it to work, change `'green'` to whatever you want. The first column of [this table](https://css-tricks.com/snippets/css/named-colors-and-hex-equivalents/) shows you all possible colors.

In [ ]:
...

<!-- BEGIN QUESTION -->

After creating your visualization above, run the following cell. You should see a picture of your graph. You must run this cell in order to get credit for this question.

<!--
BEGIN QUESTION
name: q2b
points: 2
manual: true
-->

In [ ]:
# Run this cell, don't change anything.
save_and_show(fig_2b, 'images/saved/2b.png')

<!-- END QUESTION -->



### Question 2c

Wouldn't it be nice if we could visualize airports on, say, a map? That would require knowing the latitude and longitude of each airport, which is not information we currently have.

Run the following cell to load a table that contains the latitudes and longitudes of most airports in the US.

In [ ]:
airports = Table.read_table('data/airports.csv') \
                .where('iata_code', are.not_equal_to('nan')) \
                .where('iso_country', 'US') \
                .select('name', 'iata_code', 'latitude_deg', 'longitude_deg')
airports

We can **join** this new table with our existing data in order to get the latitude and longitude of every one of United's domestic destinations from SFO.

In the cell below, assign `destinations` to a table with one row per unique destination from SFO on United. `destinations` should have 5 columns: `'Destination'`, `'count'`, `'name'`, `'latitude_deg'`, and `'longitude_deg'`.

The first few rows of `destinations` are given below.

| Destination   |   count | name                                             |   latitude_deg |   longitude_deg |
|--------------:|--------:|-------------------------------------------------:|---------------:|----------------:|
| ABQ           |       6 | Albuquerque International Sunport                |        35.0402 |       -106.609  |
| ANC           |      89 | Ted Stevens Anchorage International Airport      |        61.1744 |       -149.996  |
| ATL           |     660 | Hartsfield Jackson Atlanta International Airport |        33.6367 |        -84.4281 |
| AUS           |    1251 | Austin Bergstrom International Airport           |        30.1945 |        -97.6699 |
| BHM           |       2 | Birmingham-Shuttlesworth International Airport   |        33.5629 |        -86.7535 |

_Hint: We've created an intermediate table `united_grouped` for you to use in your work. If you use it, your solution should be a single method call and should fit on one line. Don't overthink this, and if you're stuck, look at the bolded word at the top of this cell._

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [ ]:
united_grouped = united.group('Destination')
destinations = ...
destinations

In [ ]:
grader.check("q2c")

<!-- BEGIN QUESTION -->

### Question 2d

Now, using the `destinations` table you just created, assign `map_2d` to a map with markers at every destination that one can travel to on United from SFO. Your map should look similar to the one below.

<img src = 'images/examples/2d.png' width = 600>

Some pointers:
1. Use `Marker.map_table`. See Lecture 28 for the type of table that `Marker.map_table` expects. In this subpart, the table you call `Marker.map_table` on should only have two columns.
2. You can set the `color` argument to be whatever you want; your markers don't need to be green.
3. Set the `marker_icon` argument to `'plane'` to make the icon of each marker an airplane.

**Note:** Remember the disclaimer at the top of the assignment. Don't forget to set `show = False`.

<!--
BEGIN QUESTION
name: q2d
points: 2
manual: true
-->

In [ ]:
map_2d = ...
map_2d

<!-- END QUESTION -->



Unlike for graphs, there is no cell to run after creating your map – just make sure your map is outputted by the cell above.

<!-- BEGIN QUESTION -->

### Question 2e

Let's add some spice to our map. Below, assign `map_2e` to a map with circles at every destination that one can travel to on United from SFO. The color of each circle should correspond to the number of flights to that airport from SFO – the more flights there are, the lighter the circle should be. Furthermore, each circle should be labeled with the name of the airport. Your map should look similar to the one below.

<img src = 'images/examples/2e.png' width = 600>

Some pointers again: 
1. You should use `Circle.map_table` instead of `Marker.map_table`.
2. As we did in Lecture 28, you will need to select columns from `destinations` and then use `relabeled` to change their names so that they match what `Circle.map_table` expects. This table should have four columns.
    - Two of its column names must be `'labels'` and `'color_scale'`.
    - We recommend assigning this table to a variable; we provide `temp_table` for this purpose, but you don't need to use it.
    - There are multiple examples in Lecture 28 that you can follow pretty closely.

3. In `Circle.map_table`, set `line_color` to `None`, `fill_opacity` to 0.7 and `None` and `area` to `500` to match our map.

**Note:** Remember the disclaimer at the top of the assignment. Don't forget to set `show = False`.

<!--
BEGIN QUESTION
name: q2e
points: 3
manual: true
-->

In [ ]:
temp_table = ...
map_2e = ...
map_2e

<!-- END QUESTION -->



Unsurprisingly, we see lighter circles at the hubs and popular West Coast destinations we discussed before.

### Question 2f

To round out our analysis, we'll look at flight delays, which we've ignored so far. The `'Delay'` column in the `united` table describes the number of minutes each flight was delayed.

As an aside, run the cell below to see a histogram of flight delays for flights to Newark (EWR) and Chicago (ORD).

In [ ]:
united.where('Destination', are.contained_in(['EWR', 'ORD'])) \
      .where('Delay', are.below(60)) \
      .hist('Delay', bins = np.arange(-25, 65, 5), group = 'Destination', density = False)

We'll now work towards creating a scatter plot describes the total number of flights and number of delayed flights from SFO to each United destination. Before creating the scatter plot, we'll first create a table `dest_with_delay`, which contains the number of flights and number of delayed flights to every destination. We say a flight is delayed if its `'Delay'`, as stored in the `united` table, is greater than 0 (positive). The first few rows of `dest_with_delay` are shown below.

| Destination   |   Number of Flights |   Number of Delayed Flights |
|--------------:|--------------------:|----------------------------:|
| ABQ           |                   6 |                           1 |
| ANC           |                  89 |                          35 |
| ATL           |                 660 |                         287 |
| AUS           |                1251 |                         390 |
| BHM           |                   2 |                           1 |

You'll recognize the first two columns of `dest_with_delay` – they're the exact same as `united_grouped`, just with `'count'` relabeled to `'Number of Flights'`. 

The third column, `'Number of Delayed Flights'`, comes from grouping `united` with a special function that we've defined for you, called `num_positive`. Run the three cells below to define it and see it in action.

In [ ]:
def num_positive(arr):
    # Returns the number of elements in arr that are greater than 0
    return len(arr[arr > 0])

In [ ]:
# 3 of these numbers are positive
num_positive(np.array([5, 1, -2, 4, 0]))

In [ ]:
# 2081 flights to EWR had a positive delay
num_positive(united.where('Destination', 'EWR').column('Delay'))

The `num_positive` function takes in an array and returns the number of elements in it that are positive. We can use it with group to determine the number of delayed flights per airline, like so:

In [ ]:
united.group('Destination', num_positive)

**Your job in the cell below** is to put together everything we just told you to create the table `dest_with_delay` that was described earlier. Almost all of the code is already provided for you – you'll just need to stitch everything together to create one table. Our solution is of the following form:

```py
dest_with_delay = united_grouped.with_columns(
      ___, ___.column(___)
).relabeled(___, 'Number of Flights')
```

<!--
BEGIN QUESTION
name: q2f
points: 2
-->

In [ ]:
dest_with_delay = ...
dest_with_delay

In [ ]:
grader.check("q2f")

### Question 2g

Now, assign `fig_2g` to a scatter plot with total number of flights on the x-axis and number of delayed flights on the y-axis. Your scatter plot should look like the example below.

<img src = 'images/examples/2g.png' width = 700>

**Note:** Remember the disclaimer at the top of the assignment. Don't forget to set `show = False`.

_Hint: This should be very straightforward, your code should fit on one line._ 

In [ ]:
fig_2g = ...
fig_2g

<!-- BEGIN QUESTION -->

After creating your visualization above, run the following cell. You should see a picture of your graph. You must run this cell in order to get credit for this question.

<!--
BEGIN QUESTION
name: q2g
points: 1
manual: true
-->

In [ ]:
# Run this cell, don't change anything.
save_and_show(fig_2g, 'images/saved/2g.png')

<!-- END QUESTION -->



### Question 2h

Now, assign `fig_2h` to a scatter plot with the same information as in the previous part, but with the following modifications:
- There are too many points in the bottom left of the graph. Only include points where `'Number of Flights'` is at least 700.
- Label each point according to the name of the corresponding airport.
- Increase the size of each point by setting `s = 50`.
- Match the title in the example below.

<img src = 'images/examples/2h.png' width = 700>

**Note:** Remember the disclaimer at the top of the assignment. Don't forget to set `show = False`.

In [ ]:
fig_2h = ...
fig_2h

<!-- BEGIN QUESTION -->

After creating your visualization above, run the following cell. You should see a picture of your graph. You must run this cell in order to get credit for this question.

<!--
BEGIN QUESTION
name: q2h
points: 2
manual: true
-->

In [ ]:
# Run this cell, don't change anything.
save_and_show(fig_2h, 'images/saved/2h.png')

<!-- END QUESTION -->



Unsurprisingly, the destinations with more flights have more delayed flights. It doesn't seem like there are any particular outliers; for most destinations, around a third of flights experience some sort of delay.

# Done!

Congrats! You've finished the second last Data 94 homework assignment!

To submit your work, follow the steps outlined on Ed. **Remember that for this homework in particular, almost all problems will be graded manually, rather than by the autograder.**

The point breakdown for this assignment is given in the table below:

| **Category** | Points |
| --- | --- |
| Autograder | 6 |
| Written (Including Visualizations) | 19 |
| **Total** | 25 |

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()